In [11]:
#!/usr/bin/env python3
"""
Bicycle demo – batched physics (20 Hz) with low-frequency UI updates (5 FPS)
Works everywhere Dash runs, requires *no* extra packages.
"""
import os, copy, numpy as np, dash
from dash import dcc, html, Input, Output, State, no_update
import plotly.graph_objects as go
from IPython.display import HTML

# ─── CORE PARAMETERS ───────────────────────────────────────────────────────────
PHYS_DT    = 0.05     # physics integration step (s)  → 20 Hz
UI_EVERY   = 4        # send data to browser every 4 phys steps
UI_DT_MS   = int(PHYS_DT * UI_EVERY * 1000)   # 200 ms → 5 FPS
FRAMES_MAX = 300

# ─── INTERACTIVE DASH WRAPPER (unchanged) ─────────────────────────────────────
class InteractiveApp:
    def __init__(self, host="0.0.0.0", port=8080, display_host="127.0.0.1"):
        self.host, self.port = host, port
        self.app = dash.Dash(__name__, suppress_callback_exceptions=True)
        self.app.layout = html.Div()           # placeholder
        self._demo_builders, self._current_demo = {}, None

        if "DEEPNOTE_PROJECT_ID" in os.environ:
            deepnote_id = os.environ["DEEPNOTE_PROJECT_ID"]
            self.url, self.is_deepnote = f"https://{deepnote_id}.deepnoteproject.com", True
        else:
            self.url, self.is_deepnote = f"http://{display_host}:{self.port}", False

    def register_demo(self, name: str, builder): self._demo_builders[name] = builder
    def use_demo(self, name: str, *a, **kw):
        if name not in self._demo_builders: raise KeyError(f"No demo named '{name}'")
        if self._current_demo is not None: self.app._callback_list = []
        self._demo_builders[name]({"app": self.app}, *a, **kw); self._current_demo = name
    def run(self):   self.app.run(jupyter_mode="external", host=self.host,
                                  port=self.port, debug=False, use_reloader=False)

# ─── BICYCLE MODEL (unchanged) ────────────────────────────────────────────────
class bot2D:
    def __init__(self): self.x = self.y = self.theta = 0; self.states = []
    def reset(self): x, y, t = self.states[0]; self.x, self.y, self.theta = x, y, t; self.states = [[x, y, t]]
    def get_state(self): return (self.x, self.y, self.theta)
    def set_state(self, x=0, y=0, theta=0): self.x, self.y, self.theta = x, y, theta; self.states.append([x, y, theta])

class Bicycle(bot2D):
    def __init__(self, length=0.25, delta_max=np.pi/3, vel_max=5):
        super().__init__(); self.length, self.delta_max, self.vel_max = length, delta_max, vel_max
        self.velocity = self.delta = 0
    def update_control(self, v, delta):
        self.velocity = np.clip(v, -self.vel_max, self.vel_max)
        self.delta    = np.clip(delta, -self.delta_max, self.delta_max)
    def drive(self, dt=PHYS_DT):
        vx, vy = self.velocity*np.cos(self.theta), self.velocity*np.sin(self.theta)
        next_x, next_y = self.x + vx*dt, self.y + vy*dt
        omega = self.velocity*np.tan(self.delta)/self.length
        next_theta = self.theta + omega*dt
        self.set_state(next_x, next_y, next_theta)
    def reset(self): super().reset(); self.velocity = self.delta = 0

# ─── DEMO UI ───────────────────────────────────────────────────────────────────
def labeled_slider(id_, label, vmin, vmax, step, value, marks):
    return html.Div(style={"display":"flex","alignItems":"center","marginBottom":"0"},
        children=[html.Label(label,style={"width":"12%","marginRight":"5%"}),
                  html.Div(dcc.Slider(id=id_, min=vmin,max=vmax,step=step,value=value,
                                      marks=marks, updatemode="drag",
                                      tooltip={"placement":"bottom"}),style={"flex":"1"})])

def bicycle_demo(ctx):
    app = ctx["app"]; bike = Bicycle()
    UIREV = "bicycle-demo"

    # ── Base figure with three traces (track, x-axis, y-axis) ────────────────
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=[bike.x],y=[bike.y],mode="lines",
                             line=dict(width=2,color="blue"),name="track"))
    fig.add_trace(go.Scatter(x=[bike.x,bike.x+1],y=[bike.y,bike.y],
                             mode="lines",line=dict(width=4,color="red"),showlegend=False))
    fig.add_trace(go.Scatter(x=[bike.x,bike.x],y=[bike.y,bike.y+1],
                             mode="lines",line=dict(width=4,color="green"),showlegend=False))
    fig.update_layout(margin=dict(l=0,r=0,t=20,b=0),
                      xaxis=dict(range=[-10,10],scaleanchor="y",scaleratio=1,title="x"),
                      yaxis=dict(range=[-10,10],title="y"),showlegend=False,uirevision=UIREV)

    # ── Layout ───────────────────────────────────────────────────────────────
    app.layout = html.Div(style={"display":"flex","flexDirection":"column",
                                 "height":"100vh"}, children=[
        dcc.Graph(id="graph",figure=fig,style={"flex":"1 1 auto"}),
        dcc.Interval(id="timer",interval=UI_DT_MS,n_intervals=0,disabled=True,
                     max_intervals=FRAMES_MAX//UI_EVERY),
        html.Div(style={"display":"flex","gap":"2rem","flexWrap":"wrap",
                        "padding":"1rem 0"}, children=[
            labeled_slider("velocity","Velocity (m/s)",-bike.vel_max,bike.vel_max,0.1,0.0,marks={}),
            labeled_slider("delta","Steering Δ (deg)",
                           -np.degrees(bike.delta_max), np.degrees(bike.delta_max),
                           1, 0.0,
                           marks={int(-np.degrees(bike.delta_max)):"-δmax",0:"0°",
                                  int(np.degrees(bike.delta_max)):"δmax"}),
            html.Div(id="frame-label",style={"alignSelf":"center","fontFamily":"monospace"}),
            html.Button("Play",  id="play-btn", n_clicks=0),
            html.Button("Pause", id="pause-btn", n_clicks=0),
            html.Button("Reset", id="reset-btn", n_clicks=0),
        ]),
    ])

    # ── Callbacks ────────────────────────────────────────────────────────────
    @app.callback(Output("timer","disabled"),
                  Input("play-btn","n_clicks"),Input("pause-btn","n_clicks"),
                  State("timer","disabled"),prevent_initial_call=True)
    def toggle_timer(n_play, n_pause, disabled):
        return dash.callback_context.triggered_id != "play-btn"

    @app.callback(Output("velocity","value"),Output("delta","value"),
                  Output("timer","disabled",allow_duplicate=True),
                  Output("graph","figure",allow_duplicate=True),
                  Output("frame-label","children",allow_duplicate=True),
                  Input("reset-btn","n_clicks"),prevent_initial_call=True)
    def reset(_):
        bike.reset()
        fresh = copy.deepcopy(fig)
        fresh.data[0].x, fresh.data[0].y = [bike.x],[bike.y]
        fresh.data[1].x, fresh.data[1].y = [bike.x,bike.x+1],[bike.y,bike.y]
        fresh.data[2].x, fresh.data[2].y = [bike.x,bike.x],[bike.y,bike.y+1]
        return 0.0,0.0,True,fresh,f"Frame: 0/{FRAMES_MAX}"

    @app.callback(Output("graph","extendData"),
                  Output("timer","disabled",allow_duplicate=True),
                  Input("timer","n_intervals"),
                  State("graph","figure"),
                  State("velocity","value"),State("delta","value"),
                  prevent_initial_call=True)
    def step(frame_idx, fig_state, v, delta_deg):
        if frame_idx >= FRAMES_MAX: return no_update, True

        # ── simulate UI_EVERY physics steps locally ──────────────────────────
        for _ in range(UI_EVERY):
            bike.update_control(v, np.radians(delta_deg))
            bike.drive(dt=PHYS_DT)

        R = np.array([[np.cos(bike.theta),-np.sin(bike.theta)],
                      [np.sin(bike.theta), np.cos(bike.theta)]])
        x_axis, y_axis = R @ np.array([1.0,0.0]), R @ np.array([0.0,1.0])

        patch = {"x":[[bike.x],
                      [bike.x, bike.x+x_axis[0]],
                      [bike.x, bike.x+y_axis[0]]],
                 "y":[[bike.y],
                      [bike.y, bike.y+x_axis[1]],
                      [bike.y, bike.y+y_axis[1]]]}
        return (patch,[0,1,2],{"x":[300,2,2],"y":[300,2,2]}), no_update

    @app.callback(Output("frame-label","children"),Input("timer","n_intervals"))
    def label(n): return f"Frame: {min(n,FRAMES_MAX)}/{FRAMES_MAX}"

# ─── LAUNCH ───────────────────────────────────────────────────────────────────
if __name__ == "__main__":
    demo = InteractiveApp(port=8080)
    demo.register_demo("Bicycle", bicycle_demo)
    demo.use_demo("Bicycle")
    demo.run()


Dash app running on http://0.0.0.0:8080/
